### Step 11.

Step 11 에서는 Function 클래스가 입력을 여러 인수를 받을 수 있도록 수정한다. 여기서는 list comprehension을 사용해서, 인수를 처리한다.

In [ ]:
import numpy as np

def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x

class Variable:
  def __init__(self, data):
    # 오로지 np.ndarray만 지원
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    # 미분값이 없을 때, 자동으로 1.0을 할당
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator)

class Function:
  def __call__(self, inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, x):
    raise NotImplementedError()

class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return (y,)

xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


### Step 11.

이번 단계에서는 Add 클래스를 좀 더 직관적으로 구현할 수 있게 수정해보도록 하겠다.

In [ ]:
class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs

    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, x):
    raise NotImplementedError()

class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return (y,)

def add(x0, x1):
  return Add()(x0, x1)

x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


In [ ]:
class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    return gy, gy

class Variable:
  def __init__(self, data):
    # 오로지 np.ndarray만 지원
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    # 미분값이 없을 때, 자동으로 1.0을 할당
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        x.grad = gx

        if x.creator is not None:
          funcs.append(x.creator)

class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.inputs[0].data
    gx = gy * 2 * x
    return gx

def square(x):
  return Square()(x)

x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


### Step 14.

위 Variable의 backward는 하나의 큰 문제를 가지고 있는데, 그것은 중복된 변수가 입력되었을 때, 역전파를 잘 계산하지 못한다는 것이다.
```python
# Variable 클래스의 backward 함수 중...

for x, gx in zip(f.inputs, gxs):
  x.grad = gx # 이 과정에서 변수가 중복되면, 가중치를 덮어씌우게 됨.
```

이번 단계에서는 이부분을 해결해본다.

In [ ]:
class Variable:
  def __init__(self, data):
    # 오로지 np.ndarray만 지원
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    # 미분값이 없을 때, 자동으로 1.0을 할당
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx # ndarray의 in-place operation 연산을 방지하기 위해 x.grad += gx 라는 표현을 쓰지 않는다.

        if x.creator is not None:
          funcs.append(x.creator)

x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print(x.grad)

3.0


그러나, 위 코드도 한가지 주의사항이 있다. 아래 코드처럼 같은 가중치를 재사용하는 연산을 여러번 하는 경우, 계속 미분값이 누적되는 문제가 생긴다.

```python
# 문제가 되는 경우
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

y = add(add(x, x), x)
y.backward()
print(x.grad)
```

따라서 Variable 클래스에 cleargrad 함수를 추가해준다.

In [ ]:
class Variable:
  def __init__(self, data):
    # 오로지 np.ndarray만 지원
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def cleargrad(self):
    self.grad = None

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          funcs.append(x.creator)

x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

x.cleargrad()
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
3.0


### Step 15.

지금까지는 한 줄로 늘어서 계산 그래프를 다뤄보았다. 그러나 지금까지의 구현은 한 줄로 늘어선 그래프보다 더 복잡한 연결의 그래프에 대한 역전파에 대해서는 잘 계산하지 못한다. 

In [ ]:
import numpy as np
import heapq

def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x

class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1

  def cleargrad(self):
    self.grad = None

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        heapq.heappush(funcs, f)
        seen_set.add(f)

    add_func(self.creator)
    while funcs:
      f = heapq.heappop(funcs)
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator)

class Function:
  def __init__(self):
    self.generation = 0
    self.inputs = None
    self.outputs = None

  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    self.generation = max([x.generation for x in inputs])
    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs if len(outputs) > 1 else outputs[0]

  # heap에서 대소비교를 위해 선언
  def __lt__(self, other):
    return self.generation > other.generation

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, x):
    raise NotImplementedError()

x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


### Step 17.

이번 단계에서는 메모리 관리에 대해서 다룬다.

> 파이썬의 인터프리터는 C언어로 구현된 CPython이다. 이번 단계에서 설명하는 파이썬 메모리 관리 설명은 CPython을 기준으로 한다.

파이썬의 메모리 관리는 두 가지 방식으로 진행된다. 다음과 같다.
* 하나는 참조(reference)수를 세는 방식이다.
* 다른 하나는 세대(generation)를 기준으로 쓸모없어진 객체를 회수하는 방식이며, 이것을 GC(Garbage Collection)으로 부른다.

#### 참조 카운트

모든 객체는 참조카운트가 0인 상태로 생성되고, 다른 객체가 참조할 때마다 1씩 증가한다. 반대로 객체에 대한 참조가 끊길 때마다 1만큼 감소하다가 0이 되면 파이썬 인터프리터가 회수해간다. 이런 방식으로 객체가 더 이상 필요 없어지면 즉시 메모리에서 삭제된다.

> 다음과 같은 경우에 참조 카운트가 증가한다.
* 대입 연산자를 사용할 때
* 함수에 인수로 전달할 때
* 컨테이너 타입 객체(리스트, 튜플, 클래스 등)에 추가할 때

다음 코드를 예시로 보자.

```python
class obj:
  pass

def f(x):
  print(x)

# a, b, c의 참조 카운트는 1이 된다.
a = obj()
b = obj()
c = obj()

# 다음 참조로 인해, b와 c이 참조 카운트는 2가 된다.
a.b = b
b.c = c

# a, b, c에 None에 대입되면서, a의 참조 카운트는 0이 되고, 결국 b, c도 지워지게 된다.
a = b = c = None
```

#### 순환 참조를 해결하기 위한 GC

다음은 순환 참조(Circular reference)를 설명하기 위해 준비한 코드입니다.

```python
a = obj()
b = obj()
c = obj()

# 아래 코드로 인해, a, b, c의 참조 카운트는 모두 2가 된다.
a.b = b
b.c = c
c.a = a

# 아래 코드로 인해, a, b, c의 참조 카운트가 1씩 줄어들지만, 0이 되는 것은 없어 메모리에 그대로 남겨지게 된다.
a = b = c = None
```

GC는 참조 카운트와 달리 메모리가 부족해지는 시점에 파이썬 인터프리터에 의해 자동으로 호출된다. 보통 순환 참조가 발생할 때, 자동으로 호출되어, 올바른 순서에 따라 메모리를 삭제한다.

In [ ]:
import weakref
import numpy as np

a = np.array([1, 2, 3])
b = weakref.ref(a)

print(b)
b()

<weakref at 0x7fb34698d650; to 'numpy.ndarray' at 0x7fb34ab69f30>


array([1, 2, 3])

In [ ]:
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1

  def cleargrad(self):
    self.grad = None

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        heapq.heappush(funcs, f)
        seen_set.add(f)

    add_func(self.creator)
    while funcs:
      f = heapq.heappop(funcs)

      # 약한 참조된 output에서 grad를 호출하기 위해 output()으로 쓴다.
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator)

class Function:
  def __init__(self):
    self.generation = 0
    self.inputs = None
    self.outputs = None

  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    self.generation = max([x.generation for x in inputs])
    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs

    # 약한 참조(weak reference)를 활용하여 순환 참조(circular reference)를 방지
    self.outputs = [weakref.ref(output) for output in outputs]
    return outputs if len(outputs) > 1 else outputs[0]

  # heap에서 대소비교를 위해 선언
  def __lt__(self, other):
    return self.generation > other.generation

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, x):
    raise NotImplementedError()

### Step 18.

머신러닝에서는 역전파로 구하고 싶은 미분값은 말단 변수(x0, x1)뿐일 때가 대부분이다. 반면 현재까지 구현한 Dezero는 중간 변수의 미분값을 모두 저장하면서 메모리를 차지하고 있다. 따라서, 중간 변수에 대해서는 미분값을 제거하는 모드를 추가하겠다.

In [1]:
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1

  def cleargrad(self):
    self.grad = None

  # retain_grad = False 이면 중간 변수에 대한 미분값이 모두 삭제된다.
  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        heapq.heappush(funcs, f)
        seen_set.add(f)

    add_func(self.creator)
    while funcs:
      f = heapq.heappop(funcs)

      # 약한 참조된 output에서 grad를 호출하기 위해 output()으로 쓴다.
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator)

      # retain_grad가 False로 설정되어 있으면, 말단 변수 외에는 미분값을 유지하지 않는다.
      if not retain_grad:
        for y in f.outputs:
          y().grad = None

다음 개선은 Function 클래스에서 self.inputs에 inputs을 저장하는 부분을 개선할 것이다. 신경망에서는 학습, 추론 두 단계가 있는데, 학습 단계에서는 backward() 함수가 역전파를 계산하기 때문에 self.inputs에 Function의 입력값을 저장해야 했지만, 추론 단계에서는 역전파 계산이 필요없기 때문에 입력값 저장이 필요하지 않다. 따라서 이부분을 개선해보겠다.

```python
import contextlib

@contextlib.contextmanager
def config_test():
  print('start')
  try:
    yield
  finally:
    print('done')

with config_test():
  print('process...')
```

`Config()` 클래스와, `@contextlib.contextmanager` 데코레이터를 사용하여, 추론 단계에서만 `self.inputs`에 `Function()` 클래스의 입력값을 저장할 수 있게 한다.

In [9]:
class Config:
  enable_backprop = True

class Function:
  def __init__(self):
    self.generation = 0
    self.inputs = None
    self.outputs = None

  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    # 역전파시에만 Function과 variable 관계를 저장
    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs])
      for output in outputs:
        output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]

    return outputs if len(outputs) > 1 else outputs[0]

  def __lt__(self, other):
    return self.generation > other.generation

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, x):
    raise NotImplementedError()

아래 코드는 `@contextlib.contextmanager` 데코레이터를 사용하는 부분이다.

In [12]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
  old_value = getattr(Config, name)
  setattr(Config, name, value)
  try:
    yield
  finally:
    setattr(Config, name, old_value)

def no_grad():
  return using_config('enable_backprop', False)

# 역전파가 필요없는 경우
with no_grad():
  x = Variable(np.array(2.0))
  y = square(x)

print(y.data)

4.0


### Step 19.

이번 단계에서는 Variable에 이름을 지정할 수 있게 함과 동시에, numpy 처럼 Variable 안에 담긴 데이터를 볼 수 있도록 `@property` 데코레이터를 활용하여 구현해 보도록 하겠다.

```python
class Variable:
  ...

  @property
  def ndim(self):
    return self.data.ndim

  @preperty
  def shape(self):
    return self.data.shape

  @preperty
  def __len__(self):
    return len(self.data)

  @preperty
  def size(self):
    return self.data.size

  def __repr__(self):
    ...

  ...
```

In [10]:
class Variable:
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.name = None
    self.grad = None
    self.creator = None
    self.generation = 0

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1

  @property
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  def __len__(self):
    return len(self.data)

  def cleargrad(self):
    self.grad = None

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        heapq.heappush(funcs, f)
        seen_set.add(f)

    add_func(self.creator)
    while funcs:
      f = heapq.heappop(funcs)

      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator)

      if not retain_grad:
        for y in f.outputs:
          y().grad = None

  def __repr__(self):
    if self.data is None:
      return 'variable(None)'

    p = str(self.data).replace('\n', '\n' + ' ' * 9)
    return 'variable(' + p + ')'

### Step 20.

이번 단계부터는 연산자 오버로드를 활용하여, `Variable` 클래스를 `numpy` 처럼 계산할 수 있도록 해보겠다.

In [2]:
class Mul(Function):
  def forward(self, x0, x1):
    y = x0 * x1
    return y

  def backward(self, gy):
    dx0 = gy * self.inputs[1]
    dx1 = gy * self.inputs[0]
    return dx0, dx1

def mul(x0, x1):
  return Mul()(x0, x1)

class Variable:
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.name = None
    self.grad = None
    self.creator = None
    self.generation = 0

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1

  @property
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  def __len__(self):
    return len(self.data)

  # 연산자 오버로드(operator overload)
  def __add__(self, other):
    return add(self, other)

  def __mul__(self, other):
    return mul(self, other)

  def cleargrad(self):
    self.grad = None

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        heapq.heappush(funcs, f)
        seen_set.add(f)

    add_func(self.creator)
    while funcs:
      f = heapq.heappop(funcs)

      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator)

      if not retain_grad:
        for y in f.outputs:
          y().grad = None

  def __repr__(self):
    if self.data is None:
      return 'variable(None)'

    p = str(self.data).replace('\n', '\n' + ' ' * 9)
    return 'variable(' + p + ')'

이제 우리는 Variable 인스턴스 a와 b가 있을 때, `a * b` 혹은 `a + b` 같은 코드로 작성할 수 있다. 하지만 안타깝게도, `a * np.array(2.0)` 처럼 ndarray 인스턴스와 함께 사용할 수는 없다. 이번 단계에서는 Variable 인스턴스와 ndarray 인스턴스, 심지어 int, float 등도 함께 사용할 수 있도록 해보겠다.

In [11]:
def as_variable(obj):
  if isinstance(obj, Variable):
    return obj
  return Variable(obj)

class Function:
  def __init__(self):
    self.generation = 0
    self.inputs = None
    self.outputs = None

  def __call__(self, *inputs):
    # inputs의 변수들을 모두 Variable로 바꾸어 준다.
    inputs = [as_variable(x) for x in inputs]
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs])
      for output in outputs:
        output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]

    return outputs if len(outputs) > 1 else outputs[0]

  def __lt__(self, other):
    return self.generation > other.generation

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, x):
    raise NotImplementedError()

class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    return gy, gy

# Variable 클래스와 float, int를 함께 계산할 수 있도록 하기
def add(x0, x1):
  x1 = as_array(x1)
  return Add()(x0, x1)

def mul(x0, x1):
  x1 = as_array(x1)
  return mul()(x0, x1)

# Variable 클래스와 float, int를 함께 계산할 수 있게 되었다.
x = Variable(np.array(2.0))
y = x + 3.0
print(y)

variable(5.0)


그러나 이러게 까지 하면 2가지 문제점이 있다. 다음과 같다.
* 첫 번째 인수가 float나 int인 경우
```python
# 2.0 에는 __mul__ 함수가 없기 때문에 오류
y = 2.0 * x
```
* 좌항이 ndarray 인스턴스인 경우
```python
# 연산자 우선순위에 의한 오류
x = Variable(np.array([1.0]))
y = np.array([2.0]) + x
```

첫번째 문제는 `__rmul__` 함수를 선언하여 해결하고, 두번째 문제는 `Variable()` 클래스의 연산자 우선순위를 조정하여 해결한다.

In [18]:
class Variable:
  # 연산자 우선순위 재설정
  __array_priority__ = 200

  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.name = None
    self.grad = None
    self.creator = None
    self.generation = 0

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1

  @property
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  def __len__(self):
    return len(self.data)

  def cleargrad(self):
    self.grad = None

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        heapq.heappush(funcs, f)
        seen_set.add(f)

    add_func(self.creator)
    while funcs:
      f = heapq.heappop(funcs)

      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator)

      if not retain_grad:
        for y in f.outputs:
          y().grad = None

  def __repr__(self):
    if self.data is None:
      return 'variable(None)'

    p = str(self.data).replace('\n', '\n' + ' ' * 9)
    return 'variable(' + p + ')'

Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

x = Variable(np.array([1.0]))
y = np.array([2.0]) + x
print(y)

variable([3.])


이번에는 Variable의 연산자 오버로드를 좀 더 다양하게 구현해보겠다.
* `__neg__`
* `__sub__`, `__rsub__`
* `__truediv__`, `__rtruediv__`
* `__pow__`

In [22]:
class Neg(Function):
  def forward(self, x):
    return -x

  def backward(self, gy):
    return -gy

def neg(x):
  return Neg()(x)

Variable.__neg__ = neg

class Sub(Function):
  def forward(self, x0, x1):
    y = x0 - x1
    return y

  def backward(self, gy):
    return gy, -gy

def sub(x0, x1):
  x1 = as_array(x1)
  return Sub()(x0, x1)

def rsub(x0, x1):
  x1 = as_array(x1)
  return Sub()(x1, x0)

Variable.__sub__ = sub
Variable.__rsub__ = rsub

class Div(Function):
  def forward(self, x0, x1):
    y = x0 / x1
    return y

  def backward(self, gy):
    x0, x1 = self.inputs[0].data, self.inputs[1].data
    gx0 = gy / x1
    gx1 = gy * (-x0 / x1 ** 2)
    return gx0, gx1

def div(x0, x1):
  x1 = as_array(x1)
  return Div()(x0, x1)

def rdiv(x0, x1):
  x1 = as_array(x1)
  return DiV()(x1, x0)

Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

class Pow(Function):
  def __init__(self, c):
    self.c = c
    super().__init__()

  def forward(self, x):
    y = x ** self.c
    return y

  def backward(self, gy):
    x = self.inputs[0].data
    c = self.c
    gx = c * x ** (c - 1) * gy
    return gx

def pow(x, c):
  return Pow(c)(x)

Variable.__pow__ = pow

# test
x = Variable(np.array(2.0))
y = x ** 3
print(y)

variable(8.0)
